# Clasificador basado en CNN

In [151]:
#Hacer imports y cargar stopwords y vectores entrenados
import os
import sys
import json
import time
import logging
import data_helper
import numpy as np
import tensorflow as tf
import pandas as pd  
import gensim
from text_cnn import TextCNN
from tensorflow.contrib import learn
from sklearn.model_selection import train_test_split

logging.getLogger().setLevel(logging.INFO)

archivoTweets = 'tweets_limpios.csv'

params = {
            "num_epochs": 500,
            "batch_size": 100,
            "num_filters": 32,
            "filter_sizes": "3,4,5",
            "embedding_dim": 128,
            "l2_reg_lambda": 0.0,
            "evaluate_every": 200,
            "dropout_keep_prob": 0.5
        }

#Cargar modelo de vectores
archivoModelo = 'Vectores.w2v'
modelo = gensim.models.FastText.load(archivoModelo)  # Cargar el modelo de vectores ya entrenado
print("Modelo de vectores cargado")

#Cargar stopwords
df = pd.read_csv("Stopwords.txt",header=None)
spanish_stopwords = df[0].values.tolist()
print("Stopwords cargados")

Modelo de vectores cargado
Stopwords cargados


In [152]:
#Método que carga los textos pre-procesados con sus polaridades y los prepara para el entrenamiento
def cargar_datos_etiquetas(filename):
    """Carga texto y polaridad"""
    df = pd.read_csv(filename)
    selected = ['Texto', 'Polaridad']
    non_selected = list(set(df.columns) - set(selected))

    df = df.drop(non_selected, axis=1) # Elimina las columnas innecesarias
    df = df.dropna(axis=0, how='any', subset=selected) # Elimina filas con valores null
    df = df.reindex(np.random.permutation(df.index)) # Revuelve el conjunto de datos

    # Convierte las polaridades en etiquetas One-hot
    labels = sorted(list(set(df[selected[1]].tolist())))
    one_hot = np.zeros((len(labels), len(labels)), int)
    np.fill_diagonal(one_hot, 1)
    label_dict = dict(zip(labels, one_hot))

    #Crea listas con los textos y las etiquetas en formato one-hot
    x_raw = df[selected[0]].tolist()
    y_raw = df[selected[1]].apply(lambda y: label_dict[y]).tolist()
    return x_raw, y_raw, df, labels

def obtener_vectores(x_raw, max_document_length):
    lista = []
    for tweet in x_raw:
        tweetVectores = []
        for palabra in tweet.split(' '):
            try:
                tweetVectores.append(modelo.wv.get_vector(palabra))
            except KeyError:
                continue
        if len(tweetVectores) > 0:
            if len(tweetVectores) < max_document_length:
                for x in range(max_document_length - len(tweetVectores)):
                    tweetVectores.append(np.zeros(params['embedding_dim']))
            lista.append(tweetVectores)
    return lista

In [159]:
def train_cnn():
    print("Paso 0: Cargar oraciones, etiquetas y parámetros")
    x_raw, y_raw, df, labels = cargar_datos_etiquetas(archivoTweets)

    print("Paso 1: Obtiene vectores de las palabras y rellena los textos para tener la misma longitud")
    max_document_length = max([len(x.split(' ')) for x in x_raw])
    print("Oración más larga tiene {} palabras. Se agregan 5 para tener espacio de sobra para nuevas oraciones".format(max_document_length))
    logging.info("Oración más larga tiene {} palabras. Se agregan 5 para tener espacio de manipulación para nuevas oraciones".format(max_document_length))
    max_document_length += 5
    x = np.array(obtener_vectores(x_raw, max_document_length))
    y = np.array(y_raw)

    print("Paso 2: Divide el dataset original en entrenamiento y prueba")
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

    print("Paso 3: revuelve el dataset de entrenamiento y divide el de entrenamiento en entrenamiento y validación")
    shuffle_indices = np.random.permutation(np.arange(len(y_train)))
    x_shuffled = x_train[shuffle_indices]
    y_shuffled = y_train[shuffle_indices]
    x_train, x_dev, y_train, y_dev = train_test_split(x_shuffled, y_shuffled, test_size=0.1)

    print("Paso 4: guarda las etiquetas en un archivo JSON: labels.json para hacer predicciones luego")
    with open('labels.json', 'w') as outfile:
        json.dump(labels, outfile, indent=4)

    logging.info('x_train: {}, x_dev: {}, x_test: {}'.format(len(x_train), len(x_dev), len(x_test)))
    logging.info('y_train: {}, y_dev: {}, y_test: {}'.format(len(y_train), len(y_dev), len(y_test)))

    print("Paso 5: Construir el grafo y el objeto CNN")
    graph = tf.Graph()
    with graph.as_default():
        session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
        sess = tf.Session(config=session_conf)
        with sess.as_default():
            cnn = TextCNN(
                sequence_length=x_train.shape[1],
                num_classes=y_train.shape[1],
                vocab_size=len(modelo.wv.vocab),
                embedding_size=params['embedding_dim'],
                filter_sizes=list(map(int, params['filter_sizes'].split(","))),
                num_filters=params['num_filters'],
                l2_reg_lambda=params['l2_reg_lambda'])

            global_step = tf.Variable(0, name="global_step", trainable=False)
            optimizer = tf.train.AdamOptimizer(1e-3)
            grads_and_vars = optimizer.compute_gradients(cnn.loss)
            train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

            timestamp = str(int(time.time()))
            out_dir = os.path.abspath(os.path.join(os.path.curdir, "trained_model_" + timestamp))

            checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
            checkpoint_prefix = os.path.join(checkpoint_dir, "model")
            if not os.path.exists(checkpoint_dir):
                os.makedirs(checkpoint_dir)
            saver = tf.train.Saver()

            # One training step: train the model with one batch
            def train_step(x_batch, y_batch):
                feed_dict = {
                    cnn.input_x: x_batch,
                    cnn.input_y: y_batch,
                    cnn.dropout_keep_prob: params['dropout_keep_prob']}
                _, step, loss, acc = sess.run([train_op, global_step, cnn.loss, cnn.accuracy], feed_dict)

            # One evaluation step: evaluate the model with one batch
            def dev_step(x_batch, y_batch):
                feed_dict = {cnn.input_x: x_batch, cnn.input_y: y_batch, cnn.dropout_keep_prob: 1.0}
                step, loss, acc, num_correct = sess.run([global_step, cnn.loss, cnn.accuracy, cnn.num_correct], feed_dict)
                return num_correct

            #Inicializa las variables del clasificador
            sess.run(tf.global_variables_initializer())

            # Training starts here
            train_batches = data_helper.batch_iter(list(zip(x_train, y_train)), params['batch_size'], params['num_epochs'])
            best_accuracy, best_at_step = 0, 0

            print("Paso 6: entrenar el modelo de CNN con x_train y y_train (batch por batch)")
            for train_batch in train_batches:
                x_train_batch, y_train_batch = zip(*train_batch)
                train_step(x_train_batch, y_train_batch)
                current_step = tf.train.global_step(sess, global_step)

                if current_step % params['evaluate_every'] == 0:
                    print("Etapa: {}".format(current_stop))
                    print("Paso 6.1: evaluar el modelo con x_dev y y_dev (batch por batch)")
                    dev_batches = data_helper.batch_iter(list(zip(x_dev, y_dev)), params['batch_size'], 1)
                    total_dev_correct = 0
                    for dev_batch in dev_batches:
                        x_dev_batch, y_dev_batch = zip(*dev_batch)
                        num_dev_correct = dev_step(x_dev_batch, y_dev_batch)
                        total_dev_correct += num_dev_correct

                    dev_accuracy = float(total_dev_correct) / len(y_dev)
                    logging.critical('Exactitud en set de validación: {}'.format(dev_accuracy))

                    if dev_accuracy >= best_accuracy:
                        print("Paso 6.2: Guardar el modelo si es el mejor basado en exactitud en el set de validación")
                        best_accuracy, best_at_step = dev_accuracy, current_step
                        path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                        logging.critical('Modelo guaardado en {} en etapa {}'.format(path, best_at_step))
                        logging.critical('Mejor exactitud es {} en etapa {}'.format(best_accuracy, best_at_step))

            print("Paso 7: Predecir x_test (batch por batch)")
            test_batches = data_helper.batch_iter(list(zip(x_test, y_test)), params['batch_size'], 1)
            total_test_correct = 0
            for test_batch in test_batches:
                x_test_batch, y_test_batch = zip(*test_batch)
                num_test_correct = dev_step(x_test_batch, y_test_batch)
                total_test_correct += num_test_correct

            test_accuracy = float(total_test_correct) / len(y_test)
            logging.critical('Exactitud en set de pruebas es {} basado en el mejor modelo {}'.format(test_accuracy, path))
            logging.critical('Entrenamiento completado')

In [160]:
train_cnn()

INFO:root:Oración más larga tiene 28 palabras. Se agregan 5 para tener espacio de manipulación para nuevas oraciones


Paso 0: Cargar oraciones, etiquetas y parámetros
Paso 1: Obtiene vectores de las palabras y rellena los textos para tener la misma longitud
Oración más larga tiene 28 palabras. Se agregan 5 para tener espacio de sobra para nuevas oraciones
Paso 2: Divide el dataset original en entrenamiento y prueba
Paso 3: revuelve el dataset de entrenamiento y divide el de entrenamiento en entrenamiento y validación


INFO:root:x_train: 5845, x_dev: 650, x_test: 722
INFO:root:y_train: 5845, y_dev: 650, y_test: 722


Paso 4: guarda las etiquetas en un archivo JSON: labels.json para hacer predicciones luego
Paso 5: Construir el grafo y el objeto CNN
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



Paso 6: entrenar el modelo de CNN con x_train y y_train (batch por batch)


ValueError: Cannot feed value of shape (100, 33, 128) for Tensor 'input_x:0', which has shape '(?, 33)'

In [155]:
x_raw, y_raw, df, labels = load_data_and_labels(archivoTweets)

print(len(x_raw))
print(x_raw[0])
max_document_length = min([len(x.split(' ')) for x in x_raw])
print(max_document_length)
max_document_length = max([len(x.split(' ')) for x in x_raw])
print(max_document_length)
lista = obtener_vectores(x_raw[:10], max_document_length)
max_document_length = min([len(x) for x in lista])
print(max_document_length)
max_document_length = max([len(x) for x in lista])
print(max_document_length)
oracion = ' '.join([modelo.wv.most_similar(positive=[vector], topn=1)[0][0] for vector in lista[0]])
print(len(lista))
print(oracion)

7217
audiencia vuelve imputar bárcenas merino galeote veamos si cospedal tan buena arquitecta trillo justicia
1
28
28
28
10
audiencia vuelve imputar cenas gabino lanzarote ramos si cospedal tan buena chirigota trillo justicia xurlx xurlx xurlx xurlx xurlx xurlx xurlx xurlx xurlx xurlx xurlx xurlx xurlx xurlx


In [167]:
x_raw, y_raw, df, labels = cargar_datos_etiquetas(archivoTweets)

print("Paso 1: Obtiene vectores de las palabras y rellena los textos para tener la misma longitud")
max_document_length = max([len(x.split(' ')) for x in x_raw])
print("Oración más larga tiene {} palabras. Se agregan 5 para tener espacio de sobra para nuevas oraciones".format(max_document_length))
logging.info("Oración más larga tiene {} palabras. Se agregan 5 para tener espacio de manipulación para nuevas oraciones".format(max_document_length))
max_document_length += 5
x = np.array(obtener_vectores(x_raw, max_document_length))
x[0][0]

INFO:root:Oración más larga tiene 28 palabras. Se agregan 5 para tener espacio de manipulación para nuevas oraciones


Paso 1: Obtiene vectores de las palabras y rellena los textos para tener la misma longitud
Oración más larga tiene 28 palabras. Se agregan 5 para tener espacio de sobra para nuevas oraciones


0.9260379672050476